# A Knowledge based Recommendation System for Project Mangement Ontology learning approach

The procedure of ontology construction can be done in one of three ways: manual construction; cooperative construction (need the human intervention during the constructing process) and (semi-) automatic construction which considered the Ontology Learning (OL) approach. OL from text is the process for acquiring and representing knowledge from text [structured (database), semi-structured (XML file) and unstructured (.txt, pdf, etc)] to be in machine-understandable form [OWL, RDF (Resource Description Framework), or RDFS (Resource Description Framework Schema)], by applying a set of methods and techniques (NLP, data mining, and machine learning).

-Natural Language Processing via NLTK and Spacy matcher linguistic-based preprocessing technique: (1)Tokénization and normalization (2) part-of –speech tagging (POS), (3)posTagger, (4) stopwords, Lemmatization (Stemming), (5) chunking. levenshetein measure, TF-IDF measure, leveithen measure, cosine similarity measure, topic modeling LDA, n-gram, -Recommenadtions techniques based on ML : hiaachical clutering, classification, KNN, etc -Performance measures: Precision, Recall and F measure, -Programmation: Python, java; -Semantic web language/Tool: OWL2, RDF and SWRL

# Imports

In [1]:
import sys, fitz
import pandas as pd
import re
import string
import pandas as pd

import spacy
from spacy.matcher import Matcher
from spacy.tokens import span
from spacy import displacy


import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, RegexpParser

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 


from nltk.corpus import stopwords

from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, OWL, RDF, RDFS, SKOS, VOID, XMLNS
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.extras import describer
from rdflib.namespace import XSD


#nltk.download('conll2000')
from nltk.corpus import conll2000
from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk import ChunkParserI

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from keybert import KeyBERT


#kw_model = KeyBERT(model='all-mpnet-base-v2')


#stop = stopwords.words('english')
#nlp = spacy.load("en_core_web_sm")
#nltk.download('omw-1.4')
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

# Util Funtions For Text-Preprocessing

In [2]:
def remove_punctuation(text):
    """
        Remove the punctuation
    """
    return re.sub(r'[]!"$%&\'()*+/:;=#@?[\\^_`{|}~-]+', " ", text)

In [3]:
def remove_number(text):
    """
        Remove the number
    """
    pattern = r'[0-9]'
    # Match all digits in the string and replace them with an empty string
    new_string = re.sub(pattern, ' ', text)

    return new_string

In [4]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r' ', text)

In [5]:
def remove_lineBreak(text):
    """
        Remove line break
    """
    return re.sub("\n"," ",text)

In [6]:
def lowerCase(text):
    """
        Transform all the text to lower case
    """
    return text.lower()

In [7]:
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

# Import Data From The PDF File

## 1. Retrive The Text From The PMBOK

In [8]:
# Get The text from the pdf file with the library fitz
fdoc = fitz.open("PMBOK6-2017.pdf")
header = "Header"  # text in header
footer = "Page %i of %i"  # text in footer
page = []
for i  in range(0,573):
    page.append(fdoc[i].get_text())  # insert header

# 2. Get the text for each chapter

### 2.1 Initialize Variables

In [10]:
# variable represent each chapter that we will work with 
scope = []
schedule = []
cost = []

# variable represent title that we get from each chapter and the variable with contient the whole text that we will work with 
scope_title = []
scope_title_without_number = []
scope_text = ' '
scope_content = []

schedule_title = []
schedule_title_without_number = []
schedule_text = ' '
schedule_content = []

cost_title = []
cost_title_without_number = []
cost_text = ' '
cost_content = []

figures_scope = []
figures_schedule = []
figures_cost = []

### 2.2 Project scope management

In [11]:
for i in range(164,207):
    p = re.sub('Not For Distribution, Sale or Reproduction.', ' ', page[i])
    p = re.sub('[0-9]+\s\sPart 1 - Guide', ' ', p)
    scope.append(p)
    scope_title.append(re.findall("[0-9].*[A-Z]\n",p))
    figures_scope.append(re.findall("Figure\s[0-9]-[0-9]{1,2}.\s[A-Z]{1}.*",p))

In [12]:
for i in range(0,len(scope)):
    scope_text = scope_text + scope[i]
scope_text = scope_text + ' endchapter'

# Text Preprocessing

# Final DataFrame

# OWL File Creation

# Model Creation 